In [1]:
# change the file path here

RAMDOMSEED = 11

# original data download from BindingDB
datafile = "/home/wbm001/deeplpi/DeepLPI/data/BindingDB_All.tsv"
datapath = "/home/wbm001/deeplpi/DeepLPI/data/"

In [2]:
import pandas as pd

data = pd.read_csv(datafile,'\t',error_bad_lines=False)

b'Skipping line 1087314: expected 193 fields, saw 205\nSkipping line 1087340: expected 193 fields, saw 205\n'
b'Skipping line 1120033: expected 193 fields, saw 205\n'
b'Skipping line 1142701: expected 193 fields, saw 265\n'
b'Skipping line 1546347: expected 193 fields, saw 241\n'
b'Skipping line 1660234: expected 193 fields, saw 241\nSkipping line 1660235: expected 193 fields, saw 241\nSkipping line 1660236: expected 193 fields, saw 241\nSkipping line 1660237: expected 193 fields, saw 241\nSkipping line 1660238: expected 193 fields, saw 241\nSkipping line 1660239: expected 193 fields, saw 241\nSkipping line 1660240: expected 193 fields, saw 241\nSkipping line 1660241: expected 193 fields, saw 241\nSkipping line 1660242: expected 193 fields, saw 241\n'
b'Skipping line 1673507: expected 193 fields, saw 205\n'
b'Skipping line 1692730: expected 193 fields, saw 241\nSkipping line 1692731: expected 193 fields, saw 241\nSkipping line 1692732: expected 193 fields, saw 241\nSkipping line 169273

In [4]:
oridata = data.copy()

In [93]:
# data = oridata.copy()

In [5]:
import numpy as np

data = data.set_index("BindingDB Reactant_set_id")
data = data.drop(data["BindingDB Target Chain  Sequence.1"].dropna().index) # remove multichain sequence
print("after remove multichain sequence: " + str(len(data)))
data = data[["Ligand SMILES","Ki (nM)","Kd (nM)","BindingDB Target Chain  Sequence"]]
dataki = data[["Ligand SMILES","Ki (nM)","BindingDB Target Chain  Sequence"]].dropna() # remove data without ki and kd value
datakd = data[["Ligand SMILES","Kd (nM)","BindingDB Target Chain  Sequence"]].dropna()
print("data have ki value: " + str(len(dataki)))
print("data have kd value: " + str(len(datakd)))
dataki

after remove multichain sequence: 2291976
data have ki value: 506719
data have kd value: 93978


,Ligand SMILES,Ki (nM),BindingDB Target Chain Sequence
BindingDB Reactant_set_id,,,
1,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1,0.24,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
2,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,0.25,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
3,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,0.41,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
4,OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,0.8,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
5,OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,0.99,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
...,...,...,...
51349515,CC[C@H](C)[C@H](NC(=O)OC)C(=O)N[C@@H](Cc1ccccc...,0.560000,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...
51349516,CC[C@@H](C)[C@H](NC(=O)OC)C(=O)N[C@@H](Cc1cccc...,0.029000,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...
51349517,Cc1cccc(C)c1OCC(=O)N[C@@H](Cc1ccccc1)[C@@H](O)...,0.072000,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...


In [6]:
from math import log10


dataki["Ki (nM)"] = dataki["Ki (nM)"].map(lambda x : float(x) if str(x)[0] not in [">","<"] else float(str(x)[1:]))
datakd["Kd (nM)"] = datakd["Kd (nM)"].map(lambda x : float(x) if str(x)[0] not in [">","<"] else float(str(x)[1:]))

dataki["pki"] = dataki["Ki (nM)"].map(lambda x : -log10(x) if x != 0 else None)
datakd["pkd"] = datakd["Kd (nM)"].map(lambda x : -log10(x) if x != 0 else None)

In [7]:
# remove unproparate ki | kd

dataki = dataki.dropna()
datakd = datakd.dropna()

print("data ki remain: " + str(len(dataki)))
print("data kd remain: " + str(len(datakd)))

data ki remain: 506708
data kd remain: 93977


In [8]:
# select out the test mol and seq

from sklearn.model_selection import train_test_split

_, ki_mol_test = train_test_split(list(dataki["Ligand SMILES"]), test_size=700, random_state=RAMDOMSEED)
_, ki_seq_test = train_test_split(list(dataki["BindingDB Target Chain  Sequence"]), test_size=6, random_state=RAMDOMSEED)
_, kd_mol_test = train_test_split(list(datakd["Ligand SMILES"]), test_size=50, random_state=RAMDOMSEED)
_, kd_seq_test = train_test_split(list(datakd["BindingDB Target Chain  Sequence"]), test_size=15, random_state=RAMDOMSEED)

In [9]:
molset = set(list(dataki["Ligand SMILES"]) + list(datakd["Ligand SMILES"]))
moldict = dict(enumerate(molset))
molrev = { y:x for x,y in enumerate(molset) }

seqset = set(list(dataki["BindingDB Target Chain  Sequence"]) + list(datakd["BindingDB Target Chain  Sequence"]))
seqdict = dict(enumerate(seqset))
seqrev = { y:x for x,y in enumerate(seqset) }

In [10]:
moldict = pd.DataFrame.from_dict(moldict,orient='index')
seqdict = pd.DataFrame.from_dict(seqdict,orient='index')

In [11]:
moldict["ki_test"] = moldict[0].map(lambda x : True if x in ki_mol_test else False)
seqdict["ki_test"] = seqdict[0].map(lambda x : True if x in ki_seq_test else False)
moldict["kd_test"] = moldict[0].map(lambda x : True if x in kd_mol_test else False)
seqdict["kd_test"] = seqdict[0].map(lambda x : True if x in kd_seq_test else False)

In [12]:
dataki["mol"] = dataki["Ligand SMILES"].map(lambda x : molrev[x])
datakd["mol"] = datakd["Ligand SMILES"].map(lambda x : molrev[x])

dataki["seq"] = dataki["BindingDB Target Chain  Sequence"].map(lambda x : seqrev[x])
datakd["seq"] = datakd["BindingDB Target Chain  Sequence"].map(lambda x : seqrev[x])

/home/wbm001/miniconda3/envs/DeepPurpose/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/wbm001/miniconda3/envs/DeepPurpose/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [13]:
# drop duplicates
dataki = dataki.drop_duplicates(subset=["mol","seq"],keep='first')
datakd = datakd.drop_duplicates(subset=["mol","seq"],keep='first')
print("data have ki remain: " + str(len(dataki)))
print("data have kd remain: " + str(len(datakd)))

data have ki remain: 405985
data have kd remain: 62825


In [14]:
dataki = dataki[["mol","seq","pki"]]
dataki["state"] = 0
datakd = datakd[["mol","seq","pkd"]]
datakd["state"] = 0

In [15]:
dataki["state"] += dataki["mol"].map(lambda x : 1 if moldict.loc[x]["ki_test"] else 0)
dataki["state"] += dataki["seq"].map(lambda x : 2 if seqdict.loc[x]["ki_test"] else 0)
datakd["state"] += datakd["mol"].map(lambda x : 1 if moldict.loc[x]["kd_test"] else 0)
datakd["state"] += datakd["seq"].map(lambda x : 2 if seqdict.loc[x]["kd_test"] else 0)

In [16]:
dataki["dup"] = dataki.duplicated(subset=["mol"],keep='first') & dataki.duplicated(subset=["seq"],keep='first')
_,ki_both_test = train_test_split(np.array(dataki.loc[dataki["dup"]==True].loc[dataki["state"]==0].index), test_size=1200, random_state=RAMDOMSEED)

datakd["dup"] = datakd.duplicated(subset=["mol"],keep='first') & datakd.duplicated(subset=["seq"],keep='first')
_,kd_both_test = train_test_split(np.array(datakd.loc[datakd["dup"]==True].loc[datakd["state"]==0].index), test_size=2500, random_state=RAMDOMSEED)

dataki["state"] += dataki.index.map(lambda x : 10 if x in ki_both_test else 0)
datakd["state"] += datakd.index.map(lambda x : 10 if x in kd_both_test else 0)

In [17]:
ki_train = dataki.loc[dataki["state"]==0]
ki_test = dataki.loc[dataki["state"]!=0]
kd_train = datakd.loc[datakd["state"]==0]
kd_test = datakd.loc[datakd["state"]!=0]

In [18]:
ki_test["unseen"] = ki_test["state"].map(lambda x : "mol" if x == 1 else "seq" if x == 2 else "none" if x == 10 else "both")
kd_test["unseen"] = kd_test["state"].map(lambda x : "mol" if x == 1 else "seq" if x == 2 else "none" if x == 10 else "both")

/home/wbm001/miniconda3/envs/DeepPurpose/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/wbm001/miniconda3/envs/DeepPurpose/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
ki_train[["mol","seq","pki"]].to_csv(datapath + "ki_train.csv")
kd_train[["mol","seq","pkd"]].to_csv(datapath + "kd_train.csv")
ki_test[["mol","seq","pki","unseen"]].to_csv(datapath + "ki_test.csv")
kd_test[["mol","seq","pkd","unseen"]].to_csv(datapath + "kd_test.csv")

In [ ]:
moldict.to_csv(datapath + "mol_dict.csv")
seqdict.to_csv(datapath + "seq_dict.csv")